<a href="https://colab.research.google.com/github/Filarh/Hello-World/blob/master/DeFooocus_Terminado.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# === Instalación y Configuración de DeFooocus ===
# Instalación de dependencias
!apt install -y aria2
!pip install pygit2==1.12.2

# === Configuración del Repositorio ===
%cd /content
!git clone https://github.com/ehristoforu/DeFooocus.git
%cd /content/DeFooocus

import os
import json
from google.colab import drive

# === Utilidades ===
def print_colored(text, color):
    """Imprime texto con color en Colab"""
    color_codes = {
        "red": "\033[91m",
        "green": "\033[92m",
        "yellow": "\033[93m",
        "blue": "\033[94m",
        "magenta": "\033[95m",
        "cyan": "\033[96m",
        "white": "\033[97m",
        "reset": "\033[0m"
    }
    print(f"{color_codes.get(color, color_codes['reset'])}{text}{color_codes['reset']}")

def modificar_json(nombre_archivo: str, link_archivo: str, aspect_ratio: str, formato: str):
    """Modifica o crea archivo JSON de configuración"""
    carpeta = '/content/DeFooocus/presets'
    archivos = os.listdir(carpeta)

    json_encontrado = 'default.json' if 'default.json' in archivos else next((archivo for archivo in archivos if archivo.endswith('.json')), None)

    if not json_encontrado:
        print_colored("Error: No se encontraron archivos JSON en la carpeta de presets.", "red")
        return False

    try:
        with open(os.path.join(carpeta, json_encontrado)) as file:
            data = json.load(file)

        archivo_modelo = f"{nombre_archivo}.{formato}"
        data.update({
            'default_model': archivo_modelo,
            'default_aspect_ratio': aspect_ratio,
            'default_refiner': "",
            'checkpoint_downloads': {archivo_modelo: link_archivo}
        })

        nuevo_nombre = f"{nombre_archivo}.json"
        with open(os.path.join(carpeta, nuevo_nombre), 'w') as file:
            json.dump(data, file, indent=2)

        print_colored(f"✓ Configuración guardada en {nuevo_nombre}", "green")
        return True
    except Exception as e:
        print_colored(f"Error al modificar JSON: {e}", "red")
        return False

def download_model_aria2(url: str, nombre_archivo: str, formato: str, tokenCIVITAI: str = "", tokenHUGGINGFACE: str = ""):
    """Descarga modelo usando aria2 con soporte para tokens de autenticación"""
    try:
        model_dir = "/content/DeFooocus/models/checkpoints"
        os.makedirs(model_dir, exist_ok=True)

        archivo_modelo = f"{nombre_archivo}.{formato}"
        destino = os.path.join(model_dir, archivo_modelo)

        if os.path.exists(destino):
            print_colored(f"✓ Modelo {archivo_modelo} ya existe", "cyan")
            return destino

        # Agregar tokens de autenticación si es necesario
        if "huggingface" in url and tokenHUGGINGFACE:
            url += f"&token={tokenHUGGINGFACE}"
        elif "civitai" in url and tokenCIVITAI:
            url += f"&token={tokenCIVITAI}"

        print_colored("⟳ Iniciando descarga del modelo...", "blue")
        os.system(f'aria2c -x 16 -s 16 -k 1M "{url}" -d "{model_dir}" -o "{archivo_modelo}"')

        if os.path.exists(destino):
            print_colored(f"✓ Modelo descargado en: {destino}", "green")
            return destino

        print_colored("✗ Error: No se pudo descargar el modelo", "red")
        return None

    except Exception as e:
        print_colored(f"✗ Error durante la descarga: {e}", "red")
        return None

# === Parámetros Configurables ===
# @title 🔧 Configuración General
# @markdown ### Configuración de Drive y Directorios
activar_drive = True  # @param {type:"boolean"}
# @markdown ### Configuración del Modelo
modificar_modelo = True  # @param {type:"boolean"}
nombre_archivo = "Riot"  # @param {type:"string"}
link_archivo = "https://civitai.com/api/download/models/181589"  # @param {type:"string"}
aspect_ratio = "1024*1024"  # @param ["512*512", "768*768", "1024*1024", "1024*576"] {type:"string"}
formato = "safetensors"  # @param ["safetensors", "ckpt"] {type:"string"}
# @markdown ### Tokens de Autenticación (Opcional)
tokenCIVITAI = ""  # @param {type:"string"}
tokenHUGGINGFACE = ""  # @param {type:"string"}
# @markdown ### Configuración de la Interfaz
theme = "dark"  # @param ["dark", "light"] {type:"string"}
preset = "default"  # @param ["default", "custom"] {type:"string"}

# === Montaje de Google Drive ===
if activar_drive:
    drive.mount('/content/drive')
    nuevo_dir_lora = '/content/drive/MyDrive/MyLoras'
    nuevo_dir_salida = '/content/drive/MyDrive/Fooocus_output'
    os.makedirs(nuevo_dir_lora, exist_ok=True)
    os.makedirs(nuevo_dir_salida, exist_ok=True)

    launch_py_path = '/content/DeFooocus/launch.py'
    if os.path.exists(launch_py_path):
        with open(launch_py_path, 'r') as file:
            launch_py_content = file.read()

        codigo_drive = f'''
import os
import json

nuevo_dir_lora = '{nuevo_dir_lora}'
nuevo_dir_salida = '{nuevo_dir_salida}'

config_path = os.path.abspath("./config.txt")
config_dict = {{}}

if os.path.exists(config_path):
    with open(config_path, "r", encoding="utf-8") as json_file:
        config_dict = json.load(json_file)

config_dict['path_loras'] = os.path.abspath(nuevo_dir_lora)
config_dict['path_outputs'] = os.path.abspath(nuevo_dir_salida)

with open(config_path, "w", encoding="utf-8") as json_file:
    json.dump(config_dict, json_file, indent=4)
'''
        if codigo_drive not in launch_py_content:
            with open(launch_py_path, 'w') as file:
                file.write(codigo_drive + launch_py_content)
            print_colored("✓ Configuración de Drive actualizada", "green")

# === Procesamiento Principal ===
if modificar_modelo:
    if modificar_json(nombre_archivo, link_archivo, aspect_ratio, formato):
        model_sd = download_model_aria2(link_archivo, nombre_archivo, formato, tokenCIVITAI, tokenHUGGINGFACE)
        if not model_sd:
            print_colored("✗ Error en la configuración del modelo", "red")
            exit(1)

# === Iniciar DeFooocus ===
print_colored("\n=== Iniciando DeFooocus ===", "magenta")
advanced_args = "--share --attention-split --always-high-vram --disable-offload-from-vram --all-in-fp16"
launch_args = f"{advanced_args} --theme {theme}"
if preset != "default":
    launch_args += f" --preset {preset}"

!python entry_with_update.py {launch_args}

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
aria2 is already the newest version (1.36.0-1).
0 upgraded, 0 newly installed, 0 to remove and 49 not upgraded.
/content
fatal: destination path 'DeFooocus' already exists and is not an empty directory.
/content/DeFooocus
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
✓ Configuración guardada en Riot.json
⟳ Iniciando descarga del modelo...
